This notebook serves as a learning point to understand the business model of flights and determining the trends of the business aspect of airlines and travel industry. Data set is used from synthetic souce found here on kaggle.

Part 1 - An overview of the company's sales. Used to determined business question. 

Part 2 - (To be decided and added)

## Part 1 - Company Overview

### Import Libraries and Data from Input

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))     
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from matplotlib import cm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier
def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)
from sklearn.metrics import mean_squared_error, mean_absolute_error, average_precision_score, f1_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score, confusion_matrix,classification_report


In [2]:
df_user = pd.read_csv('/kaggle/input/argodatathon2019/users.csv')
df_hotel = pd.read_csv('/kaggle/input/argodatathon2019/hotels.csv')
df_flight = pd.read_csv('/kaggle/input/argodatathon2019/flights.csv')

## Verifying df_user

In [3]:
df_user.isnull().sum()

In [4]:
df_user.head()

In [5]:
df_user.groupby("company").size()

The company is working with four different companies. 4You has the most customers making this the biggest partner that the company has.

##  Verifying df_flight

In [6]:
df_flight.head()

In [7]:
df_flight.groupby("agency").size()

The agency has three different flights with FlyingDrops with the least amount. 

In [8]:
df_flight.groupby(['travelCode', 'userCode']).size().max()

The max amount of travel code and user code meaning that the max amount of bookings is a round trip ticket

In [9]:
df_flight.groupby(['travelCode', 'userCode']).size().min()

With the minimum amount of user and travel code is also 2. Thats means that all the bookings created are round trip tickets. 

In [10]:
df_flight["datetime"] = pd.to_datetime(df_flight["date"])
df_flight['monthYear'] = pd.to_datetime(df_flight['datetime']).dt.to_period('M')
df_flight['year'] = pd.to_datetime(df_flight['datetime']).dt.year
df_flight['month'] = pd.to_datetime(df_flight['datetime']).dt.month

In [11]:
print("Dataset ranges from ", df_flight["datetime"].min(), "to", df_flight["datetime"].max())

## The data is ranged from September 2019 onto 2023, we can evaluate the data based on these time frames with a picture of the grow rate

In [12]:
df_month_grow_rate = df_flight.groupby('monthYear')["price"].sum().to_frame()
#data within 09-2019 is not complete. Dropping the data for ease of use and time for cleaning
df_month_grow_rate = df_month_grow_rate .iloc[1: , :]
df_month_grow_rate['lastMonthPrice'] = df_month_grow_rate['price'].shift(periods=1)
# Calculate the month growth rate
df_month_grow_rate['monthGrowthRate'] = (df_month_grow_rate['price']-df_month_grow_rate['lastMonthPrice'])/df_month_grow_rate['lastMonthPrice']
# to show percentage without decimal
df_month_grow_rate['MoM(%)'] = round((df_month_grow_rate['monthGrowthRate']),3) * 100 
df_month_grow_rate = df_month_grow_rate .reset_index()
df_month_grow_rate['MoM(%)'] = round(round((df_month_grow_rate['monthGrowthRate']),3) *100,2)
df_month_grow_rate['monthYearInt'] = df_month_grow_rate['monthYear'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)

In [13]:
fig1, ax1 = plt.subplots(figsize=(15, 4))
fig2, ax2 = plt.subplots(figsize=(15, 4))
df_month_grow_rate.groupby('monthYear')["price"].sum().plot(kind='line', ax=ax1, marker="o")
df_month_grow_rate.groupby('monthYearInt')["MoM(%)"].sum().plot(
    kind='bar', ax=ax2,color="purple", alpha=0.3, label=True)

for a, b, in zip(range(45),df_month_grow_rate["MoM(%)"]):
    if b < 0:
        plt.text(a-0.5, b, str(b) + '%', rotation=0, fontsize='small')
    if b > 0:
        plt.text(a-0.5, b, str(b) + '%', rotation=0, fontsize='large', c = 'red')
    
fig1.suptitle('Monthly Price Sum', fontsize = '20')
fig2.suptitle('MoM', fontsize = '20')

ax1.set(xlabel=' ', ylabel='Price Sum')
ax2.set(xlabel='Year/Month', ylabel='Mom(%)')

The price decreases with each momth starting from October 2019. However, during the Spring and Summer months, the price will spike anywhere from ~7-10%. The company will want to take advantage of the holiday seasons. But as you can see the level off near January at a steady decline. With customers dropping off quickly. 

In [14]:
df_flight_data = df_flight.loc[(df_flight['monthYear'] >= '2019-10') & (df_flight['monthYear'] <= '2021-09')]
data = df_flight_data.groupby(['year','monthYear','month'], as_index=False)['price'].sum()
#data['index'] = np.resize(np.arange(1,12),2*12)
data['index'] = [1,2,3,4,5,6,7,8,9,10,11,12,1,2,3,4,5,6,7,8,9,10,11,12]
data['group'] = ['A','A','A','A','A','A','A','A','A','A','A','A','B','B','B','B','B','B','B','B','B','B','B','B']
    

In [15]:
fig, axs = plt.subplots(figsize=(12, 4))
sns.lineplot(data=data, x='index', y='price', hue='group',markers= ["o","<"],palette = 'hot')
axs.set_xticks(range(1,13))
axs.set_xticklabels(['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'],fontsize = 17)
axs.set_xlabel("Month", fontsize = 25)
axs.set_ylabel("Price Sum", fontsize = 25)
plt.legend(title='GROUP', loc='upper right', labels=['A 2019 Oct - 2020 Sep', 'B 2020 Oct - 2021 Sep'],fontsize = 12)
plt.show()

You can see that the revenue of the Price Sum falls, as groups A and B have the same patterns with an uptick around December-January, and June going into July. A possile reasoning is more vacation and time off during those times. 

By divding the groups between the two years you can see the patterns but also a question. 

### Why Does group B (October 2020) have a lower revenue than the previous Group A (October 2019)? 

We can see that do more analysis to gain more insight
1. Percentage of new customers. 
2. How did customers travel? Just flights or was there something more? 
3. Which features are more related to customer retention? 

## Part 2 - Coming Soon